# Introducción a Redis
## Author: Misael López Sánchez, FCiencias UNAM

En este módulo nos introducimos a lo que es trabajar con redis desde Python 3.
Algunos links interesantes que se usaron se encuentran aquí:
* <a href="https://redis.io/commands#set">  Coomand Redis </a>
* <a href="https://realpython.com/python-redis/"> Python-Redis </a>

### Introducición.
Lo primero que realizamos fue instalar redis en nuestro ordenador con las siguentes instrucciones en la línea de comandos de Ubuntu:
<img src="redis_inicio.jpeg" style="with: 300px;"/>

Y proseguimos a instalar todas las dependencias posibles y ver que funcionaba adecuadamente:

<img src="redis_cli.jpeg" style="with: 300px;"/>

Una vez operada la base de datos pasamos a instalar las bibliotecas necesarias para trabajar con Python y redis, en este caso trabajamos con dos bibliotecas: 

* redis: Para crear la conexión entre Python y Redis.
* os: Para trabajar con comandos de entrada por consola en Python.


#### Instalamos redis

pip install redis

In [1]:
# Importamos bibliotecas que posteriormente vamos a utilizar
import redis as rds
import os

Creamos la conexión entre Python y Redis.
Modificamos el archivo redis.conf para que pidiera una contraseña antes de iniciar a trabajar: lcd-2019-M

In [6]:
# Checamos que la conexion con redis esté funcionando correctamente.
redis1 = rds.Redis(host='localhost',port=6379)
redis2 = rds.Redis(host='localhost',port=6379)

In [7]:
# Comprobamos la conexión con la base de datos
ping = redis1.ping
ping

<bound method Redis.ping of Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>>

In [8]:
ping2 = redis2.ping
ping2

<bound method Redis.ping of Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>>

In [9]:
redis1.set('test','examen')

True

In [10]:
redis1.get('test')

b'examen'

In [11]:
redis2.set('hola','adios')

True

In [12]:
redis2.get('hola')

b'adios'

Estas funciones sirven como auxiliares, la principal se encuentra un poco más adelante.

In [13]:
os.system('cls')
clear = lambda:os.system('cls')

def menu_inputs():
    """
    @Author: MLS y JJTL
    summary: Menu que imprime la interfaz grafica para ser un puente entre Python y Redis.
    Parameters: NULL
    Returns: Pantalla de la interfaz grafica
    """
    print("")
    print ('+'*20," Opciones ", '+'*20, '\n')
    print("0.- Agregar Registro")
    print("1.- Borrar Registro")
    print("2.- Ver todos los registros de un usuario")
    print("3.- Ver URL o Nickname de un usuario")
    print("4.- Ver la Wishlist")
    print("5.- Salir")
    
def menu_inputs_4_3():
    """
    @Author: MLS y JJTL
    summary: Menu para ver URL o Nickname de un usuario.
    Parameters: NULL
    Returns: Pantalla de la interfaz grafica
    """
    print("")
    print ('+'*20," Opciones ", '+'*20, '\n')
    print("0.- Ingrese el URL para ver su Nickname")
    print("1.- Ingrese el Nickname para ver su URL")
    print("2.- Salir")

def op4_3():
    while True:
        menu_inputs_4_3()
        optionM = input("¿Qué desea hacer? >> ")
        #clear()
        if optionM == '0': #Agregar Registro
            id = int(input("Ingrese el identificador: "))
            url = str(input("Ingrese el URL: "))
            print("El nickname del URL de este usuario es: ", redis2.hmget(id,'URL'))
            
        elif optionM == '1':
            id = int(input("Ingrese el identificador: "))
            nickname = str(input("Ingrese el Nickname: "))
            print("El URL del nickname de este usuario es: ", redis2.hmget(id,'Nickname'))
        
        elif optionM == '2':
            break;
            
        else:
            print("Opción no válida. Por favor, Vuelva a intentarlo.")
        

def op4():
    while True:
        menu_inputs()
        optionM = input("¿Qué desea hacer? >> ")
        #clear()
        if optionM == '0': #Agregar Registro
        
            id = str(input("Ingrese el identificador: "))
            url = input("Ingrese la URL: ")
            nickname = input("Ingrese un nickname al URL: ")
            clasificacion = input("Ingrese la clasificación: ")
            categoria = input("Ingrese la categoría: ")
        
            #Creamos los conjuntos para cada uno de los datos de los usuarios.
            redis1.sadd('URL:'+id,url)
            redis1.sadd('Nickname:'+id,nickname)
            redis1.sadd('Clasificacion:'+id,clasificacion)
            redis1.sadd('Categoria:'+id,categoria)
            # Creamos Hashes solo para obtener posteriormente el url, dado el 
            # nickname o viceversa (redis2 solo se encarga de esto)
            # Esto es, el valor de la llave URL es el nickname y el valor de la 
            # llave nickname es el URL
            redis2.hmset(id,{'URL':nickname,'Nickname':url})
            
            #Datos posibles de ingreso: Si - para agregar a wishlist; No caso Contrario
            wish = str(input("¿Desea agregar el registro a su Wishlist?\n"))
            
            if(wish == 'Si'):
                redis1.sadd('URLW:'+id,url)
                redis1.sadd('NicknameW:'+id,nickname)
                redis1.sadd('ClasificacionW:'+id,clasificacion)
                redis1.sadd('CategoriaW:'+id,categoria)
                
            
            
            
        elif optionM == '1': #Borrar registro
            id = input("Ingrese el identificador: ")
            url = input("Ingrese la URL: ")
            nickname = input("Ingrese un nickname al URL: ")
            clasificacion = input("Ingrese la clasificación: ")
            categoria = input("Ingrese la categoría: ")
            
            redis1.srem('URL:'+id,url)
            redis1.srem('Nickname:'+id,nickname)
            redis1.srem('Clasificacion:'+id,clasificacion)
            redis1.srem('Categoria:'+id,categoria)
            
            wish = str(input("¿Desea removerlos también de la wishlist?\n"))
            
            if(wish == "Si"):
                redis1.srem('URLW:'+id,url)
                redis1.srem('NicknameW:'+id,nickname)
                redis1.srem('ClasificacionW:'+id,clasificacion)
                redis1.srem('CategoriaW:'+id,categoria)
                
            
        elif optionM == '2': #Ver registros, los cuales se van asignando en diferentes 
            # variables dependiendo el id asignado.
            id = int(input("Ingrese el identificador: "))
            
            print("Los datos para este usuario son: ")
            sURL = redis1.smembers(f'URL:{id}')
            sNickname = redis1.smembers(f'Nickname:{id}')
            sClasificacion = redis1.smembers(f'Clasificacion:{id}')
            sCategoria = redis1.smembers(f'Categoria:{id}')
            
            #Imprimimos los registros con forme se fueron insertando
            print(f'Los URLs son: {sURL} ') 
            print(f'Los nickname son: {sNickname}')      
            print(f'Las clasificaciones son: {sClasificacion}')
            print(f'Las categorías son: {sCategoria}')
            
        elif optionM == '3': # Ver URL o Nickname de un usuario
            op4_3()
            
        elif optionM == '4': #Presentar todos los registros de la wishlist
            
            id = int(input("Ingrese el identificador: "))
            
            print("Los datos para este usuario son: ")
            sURL = redis1.smembers(f'URLW:{id}')
            sNickname = redis1.smembers(f'NicknameW:{id}')
            sClasificacion = redis1.smembers(f'ClasificacionW:{id}')
            sCategoria = redis1.smembers(f'CategoriaW:{id}')
            
            #Imprimimos los registros con forme se fueron insertando
            print(f'Los URLs son: {sURL} ') 
            print(f'Los nickname son: {sNickname}')      
            print(f'Las clasificaciones son: {sClasificacion}')
            print(f'Las categorías son: {sCategoria}')
            
            
        elif optionM == '5':
            break;
        
        else:
            print("Opción no válida. Por favor, Vuelva a intentarlo.")
        
            
            
        input("\n\n Presione ENTER para continuar.")
        clear()
         

Esta aplicación funciona como un menú y está conformado  por 6 opciones; recibe los datos de los usuarios y los guarda en datos de la siguiente manera:
<img src="redis5.png" style="width: 300px;">

Cabe mencionar que usa estructuras de datos diferentes:
* Usuarios.
* URL, nickname, clasificación y categoría son guardados en conjuntos, porque en ellos es fácil poder realizar operaciones clásicas de conjuntos.

In [14]:
# Funciones auxiliares para ingresar datos por consola en Python.
os.system('cls')
clear = lambda:os.system('cls')

# Observación: La aplicación no permite acentos ni comas, por lo que al registrarlas en Redis se deforman.

def menu_aplication():
    """
    @Author:Mls21
    summary: Menú que imprime la interfaz gráfica para ser un puente entre Python y Redis.
    Parameters: NULL
    Returns: Pantalla de la intefaz gŕafica
    """
    print("")
    print ('+'*20," MENU ", '+'*20, '\n')
    print ("0.- Ingresar un nuevo usuario")
    print ("1.- Editar datos de un usuario")
    print ("2.- Buscar usuario")
    print ("3.- Cerrar sesión de un usuario")
    print ("4.- Registros")
    print ("5.- Comparar registros")
    print ("6.- Salir")
    print ("")

        
# Empezamos con la interfaz gráfica y todo su desarrollo    
while True:
    # Inicializamos llamando al menú y eligiendo una opción
    menu_aplication()
    opMenu = input("Ingrese una opción >> ")
    clear()
    
    if opMenu == '0': # Ingresar un nuevo usuario
        print()
        print('-'*5," Ingrese el usuario:", '-'*5, '\n')
        # Ingresar un id para identificar; Núm. Cuenta para hacerlo práctico
        id = str(input("Identificador: "))
        # Nombre del usuario
        usr = input("Usuario: ")
        # Ingresamos una ciudad: Nada más para hacer bulto
        ciudad = input("Ciudad: ")
        redis1.hmset(id, {'user':usr, 'ciudad':ciudad})
        
    elif opMenu == '1': # Editar datos de usuario
        print()
        print('-'*5, "Editar datos de usuarios", '-'*5, '\n')
        id = str(input("Ingrese el identificador del usuario existente: "))
        # Ingresamos los comandos para ver si es que el usuario existe
        if (redis1.scan(0, id)):
            usr = input("Nuevo usuario: ")
            ciudad = input("Nueva ciudad: ")
            redis1.hmset(id, {'user': usr, 'ciudad':ciudad})
        else:
            print('El usuario no existe.')
            
    elif opMenu == '2': # Buscar Usuario.
        # En caso de hayamos cerrado la sesion de un usuario
        # o que el identificador no exista, usamos un try -
        # except para indicar que no se encuentra el deter-
        # minado identificador.
        try:
            print()
            print('-'*10, 'Buscador', '-'*10, '\n')
            id = str(input('Ingrese el identificador: '))
            print(redis1.hgetall(id), '\n')
            print('Usuario: ', redis1.hget(id, 'user').decode())
            print('Ciudad: ', redis1.hget(id, 'ciudad').decode())
        except AttributeError:
            print("El identificador que usted ingresó no se encuentra.")
                
        
    elif opMenu == '3': #Cerrar sesión de usuario
        print()
        print('-'*5, "Cerrar Sesión de un usuario", '-'*5, '\n')
        id = str(input("Identificador: "))
        if (redis1.scan(0,id)):
            redis1.delete(id) #borramos todos los urls de este usuario y así sucesivamente.
            redis1.delete('URL:'+id)
            redis1.delete('Nickname:'+id)
            redis1.delete('Categoría:'+id)
            redis1.delete('Clasificación:'+id)
        else: 
            print("El usuario no existe.")
    
    elif opMenu == '4': # Ingresar registros
        # Iniciamos a trabajar con los registros los cuales los guardaremos en sets porque son manipulables
        op4()
        
    elif opMenu == '5': # Comparamos los registros para encontrar matches comúnes
        print()
        usuario1 = str(input("Ingrese el identificador del usuario1: "))
        usuario2 = str(input("Ingrese el identificador del usuario2: "))
        # Creamos un nuevo conjunto en el cual guardaremos los datos de la intersección
        int_dato = 'Intersección' + usuario1 + usuario2
        
        # Definimos los conjuntos delas intersecciones
        url_inter = redis1.sinterstore('URLW'+int_dato,'URLW:'+usuario1,'URLW:'+usuario2)
        nick_inter = redis1.sinterstore('NicknameW'+int_dato, 'NicknameW:'+usuario1, 'NicknameW:'+usuario2)
        cate_inter = redis1.sinterstore('CategoriaW'+int_dato, 'CategoriaW:'+usuario1, 'CategoriaW:'+usuario2)
        clasif_inter = redis1.sinterstore('ClasificacionW'+int_dato, 'ClasificacionW:'+usuario1, 'ClasificacionW:' + usuario2)
        
        # Mandamos a llamar el nuevo conjunto donde se almacenaron las intersecciones
        sURL = redis1.smembers(f'URLW{int_dato}')
        sNickname = redis1.smembers(f'NicknameW{int_dato}')
        sClasificacion = redis1.smembers(f'ClasificacionW{int_dato}')
        sCategoria = redis1.smembers(f'CategoriaW{int_dato}')
            
        #Imprimimos los registros comunes para todos los registros registrados por cada uno de los usuarios    
        print(f'Los URLs comunes son: {sURL} ') 
        print(f'Los nicknames comunes son: {sNickname}')      
        print(f'Las clasificaciones comunes son: {sClasificacion}')
        print(f'Las categorías comunes son: {sCategoria}')
        
    elif opMenu == '6':
        #Terminamos la ejecución del programa
        break;
    
    else:
        print("Opción no válida. Por favor, Vuelva a intentarlo.")
        menu_aplication()
        
    
    input("\n\n Presione ENTER para continuar.")
    clear()
     


++++++++++++++++++++  MENU  ++++++++++++++++++++ 

0.- Ingresar un nuevo usuario
1.- Editar datos de un usuario
2.- Buscar usuario
3.- Cerrar sesión de un usuario
4.- Registros
5.- Comparar registros
6.- Salir

Ingrese una opción >> 2

---------- Buscador ---------- 

Ingrese el identificador: 1712
{b'user': b'jose de jesus tapia lopez', b'ciudad': b'pachuca'} 

Usuario:  jose de jesus tapia lopez
Ciudad:  pachuca


 Presione ENTER para continuar.

++++++++++++++++++++  MENU  ++++++++++++++++++++ 

0.- Ingresar un nuevo usuario
1.- Editar datos de un usuario
2.- Buscar usuario
3.- Cerrar sesión de un usuario
4.- Registros
5.- Comparar registros
6.- Salir

Ingrese una opción >> 4

++++++++++++++++++++  Opciones  ++++++++++++++++++++ 

0.- Agregar Registro
1.- Borrar Registro
2.- Ver todos los registros de un usuario
3.- Ver URL o Nickname de un usuario
4.- Ver la Wishlist
5.- Salir
¿Qué desea hacer? >> 0
Ingrese el identificador: 1712
Ingrese la URL: pilarang.unam.mx
Ingrese un nicknam



 Presione ENTER para continuar.

++++++++++++++++++++  Opciones  ++++++++++++++++++++ 

0.- Agregar Registro
1.- Borrar Registro
2.- Ver todos los registros de un usuario
3.- Ver URL o Nickname de un usuario
4.- Ver la Wishlist
5.- Salir
¿Qué desea hacer? >> 3

++++++++++++++++++++  Opciones  ++++++++++++++++++++ 

0.- Ingrese el URL para ver su Nickname
1.- Ingrese el Nickname para ver su URL
2.- Salir
¿Qué desea hacer? >> 0
Ingrese el identificador: 1712
Ingrese el URL: alejandropimentel.iimas.unam.mx
El nickname del URL de este usuario es:  [b'profpimentel']

++++++++++++++++++++  Opciones  ++++++++++++++++++++ 

0.- Ingrese el URL para ver su Nickname
1.- Ingrese el Nickname para ver su URL
2.- Salir
¿Qué desea hacer? >> 1
Ingrese el identificador: 1712
Ingrese el Nickname: profpimentel
El URL del nickname de este usuario es:  [b'alejandropimentel.iimas.unam.mx']

++++++++++++++++++++  Opciones  ++++++++++++++++++++ 

0.- Ingrese el URL para ver su Nickname
1.- Ingrese el Nicknam